In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

%matplotlib inline

In [2]:
data='/Users/alejandro/AnacondaProjects/Actuary Data/train_set.csv'

In [3]:
# Load test data
# test_data = os.getcwd() + r'/Actuary Data/test_set.csv'
# df_test = pd.read_csv(test_data)

In [4]:
# load data
df = pd.read_csv(data, nrows=100000)
df.head(10)

,Row_ID,Household_ID,Vehicle,Calendar_Year,Model_Year,Blind_Make,Blind_Model,Blind_Submodel,Cat1,Cat2,...,Var5,Var6,Var7,Var8,NVCat,NVVar1,NVVar2,NVVar3,NVVar4,Claim_Amount
0,1,1,3,2005,2005,K,K.78,K.78.2,D,C,...,1.008912,0.261040,0.907793,-0.077998,M,-0.231530,-0.266117,-0.272337,-0.251419,0.0
1,2,2,2,2005,2003,Q,Q.22,Q.22.3,B,C,...,1.240851,0.432987,-0.726459,0.204785,O,-0.231530,-0.266117,-0.272337,-0.251419,0.0
2,3,3,1,2005,1998,AR,AR.41,AR.41.1,B,?,...,-0.971487,-1.405797,-0.837048,-1.176858,F,-0.231530,-0.266117,-0.272337,-0.251419,0.0
3,4,3,1,2006,1998,AR,AR.41,AR.41.1,B,?,...,-0.971487,-1.405797,-0.837048,-1.176858,F,-0.231530,-0.266117,-0.272337,-0.251419,0.0
4,5,3,2,2005,2001,D,D.20,D.20.0,J,C,...,0.812656,2.112691,1.534462,2.347260,F,-0.231530,-0.266117,-0.272337,-0.251419,0.0
5,6,3,2,2006,2001,D,D.20,D.20.0,J,C,...,0.812656,2.112691,1.534462,2.347260,F,-0.231530,-0.266117,-0.272337,-0.251419,0.0
6,7,4,1,2006,2001,AJ,AJ.129,AJ.129.3,G,C,...,0.580718,0.551128,0.416289,-0.024395,M,-0.231530,-0.266117,-0.272337,-0.251419,0.0
7,8,4,2,2006,2002,AQ,AQ.17,AQ.17.1,B,C,...,0.527193,-0.023200,-0.701884,0.226664,M,-0.231530,-0.266117,-0.272337,-0.251419,0.0
8,9,4,3,2005,2002,AQ,AQ.17,AQ.17.1,B,C,...,0.527193,-0.023200,-0.701884,0.226664,M,-0.231530,-0.266117,-0.272337,-0.251419,0.0
9,10,5,1,2005,1995,BW,BW.3,BW.3.0,D,?,...,0.176312,0.283264,0.969232,-0.792339,N,2.054683,-0.266117,-0.272337,-0.251419,0.0


In [ ]:
# print(df.columns)
# print(df_test.columns)

In [5]:
# dummy code claims
df['claim'] = df.Claim_Amount.apply(lambda x: 0 if x == 0 else 1)

In [6]:
# separate feature matrix from labels (claims in {0 = No claim, 1 = Claim})
# only continuous variables are included in the model
X = df[['Var5', 'Var6', 'Var7', 'Var8', 'NVVar1', 'NVVar2', 'NVVar3', 'NVVar4', 'Vehicle', 'Model_Year', 'Blind_Make', 'Blind_Model', 'Blind_Submodel']]
y = df[['claim', 'Claim_Amount']].values

In [7]:
# split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 42)

In [8]:
# separate feature matrix from labels (claims in {0 = No claim, 1 = Claim})
# only continuous variables are included in the model
X_train_cont = X_train[['Var5', 'Var6', 'Var7', 'Var8', 'NVVar1', 'NVVar2', 'NVVar3', 'NVVar4']].values
y = df[['claim', 'Claim_Amount']].values

In [10]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

X_train_cat = X_train[['Vehicle', 'Model_Year', 'Blind_Make', 'Blind_Model', 'Blind_Submodel']]
dummy = OneHotEncoder()

vehicle_type = dummy.fit_transform(X_train_cat[X_train_cat.columns[0]].reshape(-1, 1)).toarray()
vehicle_type = vehicle_type[:,1:]

dummyC = LabelEncoder()

Blind_Make = dummyC.fit_transform(X_train_cat[X_train_cat.columns[2]].reshape(-1, 1))
Blind_Make = dummy.fit_transform(Blind_Make.reshape(-1, 1)).toarray()
Blind_Make = Blind_Make[:,1:]

Blind_Model = dummyC.fit_transform(X_train_cat[X_train_cat.columns[3]].reshape(-1, 1))
Blind_Model = dummy.fit_transform(Blind_Model.reshape(-1, 1)).toarray()
Blind_Model = Blind_Model[:,1:]

Blind_Submodel = dummyC.fit_transform(X_train_cat[X_train_cat.columns[4]].reshape(-1, 1))
Blind_Submodel = dummy.fit_transform(Blind_Submodel.reshape(-1, 1)).toarray()
Blind_Submodel = Blind_Submodel[:,1:]

Model_Year = X_train_cat.iloc[:, 1].values.reshape(-1, 1)

X_train_cat = np.hstack((vehicle_type, Blind_Make, Blind_Model, Blind_Submodel, Model_Year))

/anaconda3/envs/cv/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  
/anaconda3/envs/cv/lib/python3.6/site-packages/ipykernel_launcher.py:11: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  # This is added back by InteractiveShellApp.init_path()
/anaconda3/envs/cv/lib/python3.6/site-packages/sklearn/preprocessing/label.py:111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/envs/cv/lib/python3.6/site-packages/ipykernel_launcher.py:15: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  from ipykernel import kernelapp as app
/anaconda3/envs/cv/lib/python3.6/site-packages/ipykernel_

In [11]:
from sklearn.decomposition import SparsePCA

pca = SparsePCA(n_components=100, method='lars')
X_train_cat_pca = pca.fit_transform(X_train_cat)
print(X_train_cat_pca.shape)
print(X_train_cat_pca.get_covariance())

KeyboardInterrupt: 

In [ ]:
# scale features to have same variance
sc = StandardScaler()
X_train_cont = sc.fit_transform(X_train_cont)
X_test = sc.transform(X_test)

In [ ]:
# fit logistic regression model
model = LogisticRegression(random_state=42)
model.fit(X_train_cont, y_train[:,0])

In [ ]:
# get in and out of sample accuracy
y_pred = model.predict(X_train_cont)
in_acc = accuracy_score(y_pred, y_train[:,0])
y_pred = model.predict(X_test)
out_acc = accuracy_score(y_pred, y_test[:,0])
print('In-sample accuracy: {}\nOut-of-sample accuracy: {}'.format(in_acc, out_acc))

In [ ]:
"""
# (1) we now have model of whether someone filed a claim or not
# (2) of those who did (claim=1), we need model of y = claim amount

Full Model:
a) given X, use (1) to assess if claim will be filed
b) if for (1), y = 1, then estimate claim amount using (2)
"""

In [ ]:
X_train_claims = X_train[y_train[:,0] == 1]
y_train_claims = y_train[y_train[:,1] != 0, 1]
y_train_claims = np.log(1 + y_train_claims)

X_test_claims = X_test[y_test[:,0] == 1]
y_test_claims = y_test[y_test[:,1] != 0, 1]
y_test_claims = np.log(1 + y_test_claims)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

In [ ]:
from sklearn.linear_model import LinearRegression

regr1 = LinearRegression()
regr1.fit(X_train_claims, y_train_claims)

# scores = cross_val_score(regr, X_claims_train, y_claims_train, cv=5)
# print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

# print(regr.feature_importances_)

#-------------------------#

# get in and out of sample accuracy
y_pred = regr1.predict(X_train_claims)
in_acc = np.sqrt(mean_squared_error(y_pred, y_train_claims))
y_pred = regr1.predict(X_test_claims)
out_acc = np.sqrt(mean_squared_error(y_pred, y_test_claims))
print('In-sample accuracy: {}\nOut-of-sample accuracy: {}'.format(in_acc, out_acc))

In [ ]:
#------------------------#
# MODEL 1: Random Forest #
#------------------------#

regr2 = RandomForestRegressor(max_depth=8, random_state=42)
regr2.fit(X_train_claims, y_train_claims)

# scores = cross_val_score(regr, X_claims_train, y_claims_train, cv=5)
# print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

print(regr2.feature_importances_)

#-------------------------#

# get in and out of sample accuracy
y_pred = regr2.predict(X_train_claims)
in_acc = np.sqrt(mean_squared_error(y_pred, y_train_claims))
y_pred = regr2.predict(X_test_claims)
out_acc = np.sqrt(mean_squared_error(y_pred, y_test_claims))
print('In-sample accuracy: {}\nOut-of-sample accuracy: {}'.format(in_acc, out_acc))